Visioneers Web Cralwer.

Insert Website in dialouge box, and content will be stored in a dictionary!

Second block of code will allow you to display content from a specific web page.

In [1]:
#Working Code

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import threading
import re
import time


# Create webcrawler class
class WebCrawler:
    def __init__(self, start_url, visiting_strategy='preorder'):
        self.start_url = start_url
        self.visiting_strategy = visiting_strategy.lower()
        self.visited_urls = set()
        self.corpus = {}
        self.main_domain = urlparse(start_url).netloc
        self.lock = threading.Lock()  # Lock for thread-safe access to shared data

    def can_crawl(self, url):
      # Check if the URL is allowed by robots.txt
      try:
          robots_url = urljoin(url, '/robots.txt')
          robots_content = requests.get(robots_url).text
          return re.search(
              r'^User-agent: \*\nDisallow: /', robots_content) is None
      except Exception as e:
          print(f"Error checking robots.txt for {url}: {e}")
          return False


    def crawl(self, url):
        if url not in self.visited_urls and self.is_same_domain(url) and self.can_crawl(url):
            print(f"Visiting: {url}")
            self.visited_urls.add(url)
            try:
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                title = soup.title.string.strip() if soup.title else 'Untitled'
                text_content = self.extract_text_content(soup)

                with self.lock:  # Thread-safe update of shared data
                    self.corpus[url] = text_content

                print(f"Text Content: {text_content[:100]}...")  # Output a snippet of text

                if self.visiting_strategy == 'preorder':
                    links = self.extract_links(soup)
                    threads = []
                    for link in links:
                        thread = threading.Thread(target=self.crawl, args=(link,))
                        threads.append(thread)
                        thread.start()
                        time.sleep(1)  # Add a delay of 1 second between requests

                    # Wait for all threads to complete
                    for thread in threads:
                        thread.join()

                # Additional visiting strategies (inorder, postorder) can be implemented here

            except Exception as e:
                print(f"Error crawling {url}: {e}")

    def extract_text_content(self, soup):
        # Extract text content only from the body of the HTML
        text_content = ' '.join([p.get_text(separator=' ', strip=True) for p in soup.body.find_all('p')])
        return text_content

    def extract_links(self, soup):
        # Extract all links from the page
        links = [link.get('href') for link in soup.find_all('a', href=True)]
        # Filter internal links only
        links = [urljoin(self.start_url, link) for link in links if link.startswith(('http', 'https'))]
        # Exclude PDF links
        links = [link for link in links if not link.endswith('.pdf')]
        # Filter out external links
        links = [link for link in links if self.is_same_domain(link)]
        # Exclude links with 'resources' in the URL
        links = [link for link in links if 'resources' not in link.lower()]
        return links

    def is_same_domain(self, url):
        return urlparse(url).netloc == self.main_domain

    def start_crawling(self):
        self.crawl(self.start_url)

    def get_crawled_data(self):
        return self.corpus

if __name__ == "__main__":
    # Get the starting URL from the user
    start_url = input("Enter the website's URL: ")

    # Instantiate the WebCrawler with the provided URL and visiting strategy
    crawler = WebCrawler(start_url=start_url, visiting_strategy='preorder')

    # Start crawling
    crawler.start_crawling()

    # Get the crawled data
    crawled_data = crawler.get_crawled_data()

    # Print the crawled data
    for url, content in crawled_data.items():
        print(f"URL: {url}")
        print(f"Content: {content[:100]}...")  # Print a snippet of content

Enter the website's URL: https://www.stjohns.edu/
Visiting: https://www.stjohns.edu/
Text Content: See how your journey aligns with what drives you. Hannah M. Queens, NY Maria Orlando, FL Jenna Charl...
Visiting: https://www.stjohns.edu/life-st-johns/career-services
Text Content: University Career Services is committed to ensuring student and alumni success. Our dedicated team o...
Visiting: https://www.stjohns.edu/about/leadership-and-administration/office-president/presidents-society
Visiting: https://www.stjohns.edu/life-st-johns/career-services/leadership-development
Text Content: Founded in 1968, the President’s Society honors those students who combine scholarship, integrity, m...
Visiting: https://www.stjohns.edu/who-we-are/faith-and-mission/campus-ministry/opportunities/plunge-program
Text Content: Developing leadership skills and giving back to the community are essential components to the educat...
Text Content: Plunges, or service immersion, are weeklong experiences where st

Text Content: International Students From 90+ Countries of Undergraduates Study Abroad A globally-minded ethos of ...


Text Content: The Center for Bankruptcy Studies is home to all of the Law School's bankruptcy initiatives, includi...
Text Content: Our Transfer Admission and Advising Counselors are readily available to assist you, and have created...Text Content: The St. John’s Center for International and Comparative Law (CICL) promotes the study of internation...Error crawling https://www.stjohns.edu/files/phd-curriculum-and-instruction-essay-prompt-instructions: 'NoneType' object has no attribute 'find_all'

Text Content: Residence Life aims to develop a residential community that supports and enhances the academic missi...

Text Content: Please click below to access the page. 8000 Utopia Parkway Queens NY 11439 718-990-2000 St. John’s U...
Text Content: Located in Queens, New York, one of the most diverse places in the world and one of the five borough...

Text Content: View the

Text Content: Thomas J. Rossi is a partner in the law firm of Rossi & Crowley, LLP concentrating his practice on A...
Text Content: St. John's University's home—New York City—received the #19 ranking on The QS Best Student Cities 20...
Error crawling https://www.stjohns.edu/sites/default/files/2023-10/School%20of%20Education%20Visiting%20Scholar%20Review%20Checklist.docx: 'NoneType' object has no attribute 'find_all'Text Content: Mark C. Niles, has joined the St. John’s Law faculty where he teaches and specializes in Civil Proce...

Text Content: Dean Michael A. Simons announced today that the Center for Law and Religion at St. John's Law will b...
Text Content: Gina M. Calabrese is a Professor of Clinical Education and the Associate Director of the Consumer Ju...
Text Content: St. John’s College of Liberal Arts and Sciences offers several graduate digital badges and one under...
Visiting: https://www.stjohns.edu/academics/programs/chemistry-bachelor-science
Visiting: https://www.stjoh

Visiting: https://www.stjohns.edu/academics/faculty/alec-p-ostrow
Visiting: https://www.stjohns.edu/academics/schools/st-johns-college-liberal-arts-and-sciences/about/non-matriculated-and-visiting-studentsVisiting: https://www.stjohns.edu/law/academics/centers-institutes/public-interest-center

Error crawling https://www.stjohns.edu/sites/default/files/2022-04/SJU%202022%20Instructions%20for%20Completing%20Summer%20Program%20Form.docx: 'NoneType' object has no attribute 'find_all'
Visiting: https://www.stjohns.edu/academics/schools/peter-j-tobin-college-business/peter-j-tobin-college-business-graduate-programs#ms
Visiting: https://www.stjohns.edu/law/jd-admissions/diversity-equity-inclusion
Visiting: https://www.stjohns.edu/about/campuses-and-locations/paris-location
Visiting: https://www.stjohns.edu/law/jd-admissions/housing#housingText Content: Manouchkathe Cassagnol, Pharm.D., BCPS, AACC Dr. Cassagnol is a tenured Associate Clinical professor...

Visiting: https://www.stjohns.edu/ac

Text Content: The first Bishop of Brooklyn, Reverend John Loughlin invited the Vincentian Fathers to establish an ...
Text Content: JUDGE BRENDAN LINEHAN SHANNON United States Bankruptcy Judge for the District of Delaware Brendan Li...
Error crawling https://www.stjohns.edu/sites/default/files/2022-04/SJU%202022%20Camp%20Emergency%20Contact%20%26%20Consent%20to%20Treat%20Form.docx: 'NoneType' object has no attribute 'find_all'
Text Content: Jeremy Cruz earned his B.A. in history from the University of California, Riverside. He studied at L...
Text Content: The Peter J. Tobin College of Business has worked with some of the premier specialists in the field ...
Text Content: Thanks for your interest in St. John's University. At St. John's, you'll find a community of entrepr...
Text Content: A dedication to diversity, equity, and inclusion is at the heart of our mission. As a Catholic and V...
Text Content:  To complete the Free Application for Federal Student Aid (FAFSA) visit studentaid.

Error crawling https://www.stjohns.edu/sites/default/files/2022-04/SJU%202022%20Meningitis%20Vaccination%20Parent%20Information%20%26%20Response%20Form.docx: 'NoneType' object has no attribute 'find_all'
Visiting: https://www.stjohns.edu/law
Visiting: https://www.stjohns.edu/law/faculty/elissa-germaine
Text Content: Jelani Jefferson Exum Appointed Next Dean of St. John’s Law Outcomes, location, community. We've got...
Visiting: https://www.stjohns.edu/academics/programs/french-bachelor-arts
Visiting: https://www.stjohns.edu/law/faculty/trent-anderson
Visiting: https://www.stjohns.edu/academics/faculty/anthony-rodriguez
Visiting: https://www.stjohns.edu/about/news/all-news?school=36
Visiting: https://www.stjohns.edu/academics/schools/peter-j-tobin-college-business/departments-faculty/faculty
Visiting: https://www.stjohns.edu/academics/programs/finance-master-science#tracksVisiting: https://www.stjohns.edu/academics/schools/st-johns-college-liberal-arts-and-sciences/programs/pathways
Vis

Visiting: https://www.stjohns.edu/academics/centers-institutes/institute-biotechnology
Visiting: https://www.stjohns.edu/about/leadership-and-administration/administrative-offices/conference-services
Error crawling https://www.stjohns.edu/sites/default/files/2020-03/M1-12853%20MA%20Chinese%20%26%20EAS.PDF: 'NoneType' object has no attribute 'find_all'
Visiting: https://www.stjohns.edu/academics/programs/global-development-and-social-justice-master-artsVisiting: https://www.stjohns.edu/academics/schools/peter-j-tobin-college-business/programs/global-programs/global-microloan-program/students/spring-2018Visiting: https://www.stjohns.edu/about/news/2023-01-24/hospitality-management-student-wins-top-award-leadership


Visiting: https://www.stjohns.edu/academics/schools/college-professional-studies/lesley-h-and-william-l-collins-college-professional-studies-centers-and-laboratories#centers-and-institutes
Visiting: https://www.stjohns.edu/admission/international-admission/international-stude

Text Content: “Education is one thing no one can take away from you.” This quote by Elin Nordegren fuels my belief...
Visiting: https://www.stjohns.edu/academics/schools/college-professional-studies/about-lesley-h-and-william-l-collins-college-professional-studies/collins-college-professional-studies-news-and-media
Error crawling https://www.stjohns.edu/files/january-2020-review-business: 'NoneType' object has no attribute 'find_all'
Text Content: St. John’s 30-credit master’s degree in Cyber and Information Security accelerates your career and t...
Text Content: Welcome to The Ellen Thrower Center for Apprenticeship and Career Services located on the Manhattan ...
Visiting: https://www.stjohns.edu/academics/faculty/erald-troja
Text Content: 96 132 48 96 48 96 Facility Room Layout Capacity Main Gym with Bleachers Fixed Bleacher & Chairback ...Visiting: https://www.stjohns.edu/academics/faculty/cynthia-chambers
Text Content: St. John’s School of Law is an accredited provider of continui

Error crawling https://www.stjohns.edu/files/2023-annual-security-and-fire-safety-report: 'NoneType' object has no attribute 'find_all'


Error crawling https://www.stjohns.edu/sites/default/files/2020-01/53R%20Brief%20%28Revised%29.PDF: 'NoneType' object has no attribute 'find_all'
URL: https://www.stjohns.edu/
Content: See how your journey aligns with what drives you. Hannah M. Queens, NY Maria Orlando, FL Jenna Charl...
URL: https://www.stjohns.edu/life-st-johns/career-services
Content: University Career Services is committed to ensuring student and alumni success. Our dedicated team o...
URL: https://www.stjohns.edu/about/leadership-and-administration/office-president/presidents-society
Content: Founded in 1968, the President’s Society honors those students who combine scholarship, integrity, m...
URL: https://www.stjohns.edu/life-st-johns/career-services/leadership-development
Content: Developing leadership skills and giving back to the community are essential components to the educat...
URL: https://www.stjohns.edu/who-we-are/faith-and-mission/campus-ministry/opportunities/plunge-program
Content: Plunges, or servic

In [2]:
#Displaying content from a specific webpage

# Assuming 'crawled_data' is the dictionary containing the crawled data
desired_url = input("Enter the website's URL: ")

# Check if the URL is present in the crawled data
if desired_url in crawled_data:
    content = crawled_data[desired_url]
    print("Content for URL:")
    print(content)
else:
    print("URL not found in crawled data.")

Enter the website's URL: https://www.stjohns.edu/academics/programs/economics-bachelor-science
Content for URL:
St. John’s economics program is designed to prepare graduates for careers in law, business, government and nonprofits. The program provides its students with the necessary skills and knowledge to investigate and critically understand the economy and economic institutions and how they shape economic behavior.  Emphasizing theoretical and empirical/analytic skills, students are prepared for various career paths.  The program also provides the historical and social context necessary to understand economic outcomes fully. Students can focus their studies on areas such as economic development, environmental economics, economic policies, and business economics. The program rewards excellence through induction into the national economics society Omicron Delta Epsilon, presenting their research at national economics conferences, participating in the Sadie Cooperative program, and com